# Clusterer

## Setup

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Import Dataset

In [2]:
import pandas as pd

file_path = "../data/catering_reviews_clustered.json"
data = pd.read_json(file_path)

if data is not None:
    print("Processed Data:")
    print(data.head())
else:
    print("No data to process.")

Processed Data:
          cateringCompanyId        cateringCompanyName  \
0  6852dce274c91954046c1d35  Le Traiteur des 4 Saisons   
1  6852d6b174c91954046c1d17          Les Petits Bruits   
2  6852dce574c91954046c1d38              Aldo Traiteur   
3  6852dd1374c91954046c1d52         La Fine Fourchette   
4  6852dcf674c91954046c1d44            Loison Traiteur   

                                             reviews  
0  [{'text': 'Nous avons fait appel au traiteur d...  
1  [{'text': 'Aurore a été super du début à la fi...  
2  [{'text': 'Le menu était très bon, nos invités...  
3  [{'text': 'Tout est dans le titre: que ce soit...  
4  [{'text': 'Nous sommes ravis d'avoir choisi Lo...  


## Prepare the prompt

In [3]:
from openai import OpenAI

# Create the client with the API key
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
# Create the prompt function
def summarize(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
  messages = [{"role": "user", "content": prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.3,
  )
  return response.choices[0].message.content

In [5]:
data["summary"] = ""
data["key_points"] = ""
data["global_score"] = ""

In [7]:
from tqdm import tqdm

def batch_reviews(reviews, batch_size=5):
    for i in range(0, len(reviews), batch_size):
        yield reviews[i:i + batch_size]

# Analyse des traiteurs
for i, row in tqdm(data.iterrows(), total=len(data)):
    company_name = row["cateringCompanyName"]
    reviews = row["reviews"]

    summaries = []
    for batch in batch_reviews(reviews, batch_size=10):
        joined = "\n\n".join([r["text"] for r in batch if "text" in r and r["text"]])
        prompt = f"""
        Tu es un assistant d'analyse d'avis pour des traiteurs de mariage.

        Voici un extrait d'avis pour le traiteur **{company_name}** :

        {joined}

        Résume les points importants en quelques phrases.
        """
        try:
            summaries.append(summarize(prompt))
        except Exception as e:
            print(f"❌ Erreur batch avec {company_name}: {e}")
            continue

    full_summary = "\n\n".join(summaries)

    # Prompt final global
    global_prompt = f"""
    Tu es un assistant d'analyse d'avis pour des traiteurs de mariage.

    Voici les résumés intermédiaires des avis pour le traiteur **{company_name}** :

    {full_summary}

    1. Fais un résumé synthétique global des avis.
    2. Détaille les points majeurs évoqués (forces, faiblesses, répétitions...).
    3. Attribue un score global subjectif sur 100 basé sur la qualité perçue.

    Donne la réponse structurée comme ceci :
    Résumé : ...
    Points clés : ...
    Score global : ...
    """

    try:
        result = summarize(global_prompt)
        parts = result.split("Score global :")
        if len(parts) == 2:
            core, score = parts
            summary_part, key_points_part = core.split("Points clés :", 1)
            data.at[i, "summary"] = summary_part.replace("Résumé :", "").strip()
            data.at[i, "key_points"] = key_points_part.strip()
            data.at[i, "global_score"] = score.strip()
        else:
            data.at[i, "summary"] = result.strip()
    except Exception as e:
        print(f"❌ Erreur analyse finale avec {company_name}: {e}")
        continue

100%|██████████| 117/117 [14:59<00:00,  7.69s/it]


In [8]:
# Sauvegarde
data.to_json("../data/catering_reviews_summary.json", orient="records", force_ascii=False, indent=2)
print("✅ Résumés sauvegardés.")

✅ Résumés sauvegardés.
